## Hypothesis Testing Demo

### School of AI - DataLit Week 2

#### Any mistakes made here are the sole property of I, Carson Bentley

This notebook is based on an article provided by Gaël Varoquaux as part of the [scipy documentation](http://scipy-lectures.org/packages/statistics/index.html#hypothesis-testing-comparing-two-groups).

See the following for a description of how the data was collected:

[Brain Size and Intelligence. Willerman et al. (1991)](https://www3.nd.edu/~busiforc/handouts/Data%20and%20Stories/correlation/Brain%20Size/brainsize.html)

In [0]:
import pandas as pd

In [0]:
URL = 'http://scipy-lectures.org/_downloads/brain_size.csv'

In [0]:
df = pd.read_csv(URL, sep=';', na_values=".", index_col=0)

In [0]:
df.head(12)

,Gender,FSIQ,VIQ,PIQ,Weight,Height,MRI_Count
1,Female,133,132,124,118.0,64.5,816932
2,Male,140,150,124,NaN,72.5,1001121
3,Male,139,123,150,143.0,73.3,1038437
4,Male,133,129,128,172.0,68.8,965353
5,Female,137,132,134,147.0,65.0,951545
6,Female,99,90,110,146.0,69.0,928799
7,Female,138,136,131,138.0,64.5,991305
8,Female,92,90,98,175.0,66.0,854258
9,Male,89,93,84,134.0,66.3,904858
10,Male,133,114,147,172.0,68.8,955466


In [0]:
df.describe()

,FSIQ,VIQ,PIQ,Weight,Height,MRI_Count
count,40.000000,40.000000,40.00000,38.000000,39.000000,4.000000e+01
mean,113.450000,112.350000,111.02500,151.052632,68.525641,9.087550e+05
std,24.082071,23.616107,22.47105,23.478509,3.994649,7.228205e+04
min,77.000000,71.000000,72.00000,106.000000,62.000000,7.906190e+05
25%,89.750000,90.000000,88.25000,135.250000,66.000000,8.559185e+05
50%,116.500000,113.000000,115.00000,146.500000,68.000000,9.053990e+05
75%,135.500000,129.750000,128.00000,172.000000,70.500000,9.500780e+05
max,144.000000,150.000000,150.00000,192.000000,77.000000,1.079549e+06


In [0]:
from scipy import stats

Check out the source code for scipy.stats [here](https://github.com/scipy/scipy/blob/master/scipy/stats/stats.py). Under the hood, scipy uses numpy for the math calculations.

### One Sample T-Test

I'm curious if the averages given by this sample vary from the standard average IQ, which I happen to know is 100. In this experiment the null hypothesis is that the population from which this sample is drawn is actually 100, and the alternative hypothesis is that it is not.

Let's use 5% as our significance, alpha

In [0]:
IQ_column_names = ['FSIQ', 'VIQ', 'PIQ']

for IQ_column in IQ_column_names:
  print(stats.ttest_1samp(df[IQ_column], 100))

Ttest_1sampResult(statistic=3.532307014238269, pvalue=0.0010766792736967715)
Ttest_1sampResult(statistic=3.3074146385401786, pvalue=0.002030117404781822)
Ttest_1sampResult(statistic=3.1030246997178783, pvalue=0.0035555593418294417)


Since the p-value is smaller than alpha, we can confidently reject the null hypothesis. That means that our average IQs of 113, 112, and 111 (for the FSIQ, VIQ, and PIQ) are most likely due to something other than random variation.

Speculating as to why these IQs are above average, I imagined a scenario in which subjects are being gathered for a data collection at a university. Many of these subjects would naturally be students.

As it turns out, this speculation was correct, as you can confirm by looking at the article linked at the top of this notebook.

### Two Sample T-Test

Suppose we want to compare the IQs of men and women.

In [0]:
groupby_gender = df.groupby('Gender')
for IQ_column in IQ_column_names:
  for gender, value in groupby_gender[IQ_column]:
    print((gender, value.mean()))

('Female', 111.9)
('Male', 115.0)
('Female', 109.45)
('Male', 115.25)
('Female', 110.45)
('Male', 111.6)


Although the males in this sample have higher average IQs in the three areas tested, these results are not necessarily significant.

Let's do a two sample t-test to see if any of these differences meet a 5% standard alpha.

In [0]:
female = df[df['Gender'] == 'Female']
male = df[df['Gender'] == 'Male']
for IQ_column in IQ_column_names:
  print(stats.ttest_ind(female[IQ_column], male[IQ_column]))

Ttest_indResult(statistic=-0.4026724743703011, pvalue=0.6894456253897778)
Ttest_indResult(statistic=-0.7726161723275011, pvalue=0.44452876778583217)
Ttest_indResult(statistic=-0.15980113150762698, pvalue=0.8738841403250049)


Since all three p-values are greater than 5%, we fail to reject the null hypothesis in each case. In other words the difference in mean IQ values for the observed data is not statistically significant.